In [22]:
import math, datetime, json
import pandas as pd
import sklearn
from sklearn import svm, preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_json("gameData.json", orient='records')

df.drop_duplicates(subset=['game_id'], keep='last', inplace=True)

In [24]:
X = df[['teamA_sval','teamA_off','teamB_sval','teamB_deff']]
# X = preprocessing.scale(X)
y = df['teamA_pts']

X2 = df[['teamB_sval','teamB_off','teamA_sval','teamA_def']]
# X2 = preprocessing.scale(X2)
y2 = df['teamB_pts']

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=64)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,random_state=64)

In [25]:
model = RandomForestRegressor(n_estimators=1000,random_state=64,criterion='mae')
model.fit(X_train, y_train)

model2 = RandomForestRegressor(n_estimators=1000,random_state=64,criterion='mae')
model2.fit(X2_train, y2_train)

RandomForestRegressor(criterion='mae', n_estimators=1000, random_state=64)

In [26]:
print(f'Model Score: {model.score(X_test,y_test)}')
print(f'Model Score: {model2.score(X2_test,y2_test)}')

Model Score: 0.3990573019329152
Model Score: 0.4506334142495564


In [81]:
predict_num = 2

a_ovr = 0.9229
a_off = 0.9651
a_def = 0.9097

b_ovr = 0.9049
b_off = 0.9513
b_def = 0.8941

if predict_num == 1:
    modelA_predict = [a_ovr, a_off, b_ovr, b_def]
    modelB_predict = [b_ovr, b_off, a_ovr, a_def]
else:
    modelA_predict = [b_ovr, b_off, a_ovr, a_def]
    modelB_predict = [a_ovr, a_off, b_ovr, b_def]

In [82]:
mpr = model.predict([ modelA_predict ])[0]
mpr2 = model2.predict([ modelB_predict ])[0]

if predict_num == 1:
    print(f"Prediction: {mpr} - {mpr2}")
else:
    print(f"Prediction: {mpr2} - {mpr}")

Prediction: 36.416 - 37.941
